## Step 4 - Climate App

Now that you have completed your initial analysis, design a Flask api based on the queries that you have just developed.

* Use FLASK to create your routes.

### Routes

* `/api/v1.0/precipitation`
  * Query for the dates and temperature observations from the last year.
  * Convert the query results to a Dictionary using `date` as the key and `tobs` as the value.
  * Return the json representation of your dictionary.

* `/api/v1.0/stations`
  * Return a json list of stations from the dataset.

* `/api/v1.0/tobs`
  * Return a json list of Temperature Observations (tobs) for the previous year

* `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`
  * Return a json list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.

  * When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.

  * When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` for dates between the start and end date inclusive.

## Hints

* You will need to join the station and measurement tables for some of the analysis queries.

* Use Flask `jsonify` to convert your api data into a valid json response object.

In [91]:
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [3]:
connection_string = 'sqlite:///hawaii.sqlite'
engine = create_engine(connection_string)

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect = True)

In [6]:
Hawaii = Base.classes.hawaii_data
session = Session(engine)

In [7]:
first_row = session.query(Hawaii).first()
type(first_row)

sqlalchemy.ext.automap.hawaii_data

In [105]:
#Query to return one year's worth of prcp data.
#Function does a groupby for each day and sums across all four stations.

#https://stackoverflow.com/questions/25558582/groupby-and-sum-in-sqlalchemy
#https://stackoverflow.com/questions/8895208/sqlalchemy-how-to-filter-date-field
prcp_query = session.query(Hawaii.prcp).filter(Hawaii.date_format >= (dt.date.today() - dt.timedelta(days=365))).all()

In [108]:
list(np.ravel(prcp_query ))

[0.0,
 0.0,
 0.31,
 2.6200000000000001,
 0.01,
 0.0,
 0.0,
 0.070000000000000007,
 0.0,
 0.0,
 0.0,
 0.0,
 0.059999999999999998,
 0.059999999999999998,
 0.01,
 0.0,
 0.029999999999999999,
 0.0,
 0.0,
 0.0,
 1.1899999999999999,
 0.72999999999999998,
 0.46999999999999997,
 0.0,
 0.34999999999999998,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.02,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.26000000000000001,
 0.01,
 0.0,
 0.02,
 0.0,
 0.02,
 0.050000000000000003,
 0.23000000000000001,
 0.32000000000000001,
 0.029999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.12,
 0.89000000000000001,
 0.26000000000000001,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01,
 0.0,
 0.0,
 0.0,
 0.050000000000000003,
 0.01,
 0.0,
 0.01,
 0.0,
 0.0,
 0.0,
 0.0,
 0.080000000000000002,
 0.13,
 0.14999999999999999,
 0.01,
 0.02,
 0.0,
 0.26000000000000001,
 0.02,

In [23]:
#Query to return the most active stations.
#http://docs.sqlalchemy.org/en/latest/orm/query.html

active_query = session.query(Hawaii.station).all()

In [26]:
df = pd.DataFrame(active_query)

In [22]:
type(list(np.ravel(active_query)))

list

In [18]:
df = pd.DataFrame(active_query, columns={'station_name', 'count'}).set_index('station_name')
df.

{'count': {'USC00511918': 1979,
  'USC00513117': 2709,
  'USC00514830': 2202,
  'USC00516128': 2612,
  'USC00517948': 1372,
  'USC00518838': 511,
  'USC00519281': 2772,
  'USC00519397': 2724,
  'USC00519523': 2669}}

In [47]:
# from datetime import datetime, timedelta
# import numpy as np
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func
# from flask import Flask, jsonify

# app = Flask(__name__)

# @app.route('/api/v1.0/stations')
# def stations():
#     results = list(np.ravel(session.query(Hawaii.name).all()))
#     return jsonify(results)

# if __name__ == "__main__":
#     app.run(debug=True)

In [ ]:
# # https://stackoverflow.com/questions/1958219/convert-sqlalchemy-row-object-to-python-dict/10370224
# #AttributeError: 'result' object has no attribute '__dict__'
# active_list = []
# for i in active_query:
#     active_list.append(i.__dict__)

In [ ]:
# #https://stackoverflow.com/questions/19110164/dumping-sqlalchemy-output-to-json
# #AttributeError: 'result' object has no attribute '__dict__'
# active_list = [i.__dict__ for i in active_query]

In [68]:
#Query to return all of the temp from 'Waihee' during 2016-08-23 and 2017-08-23.

query_temp = session.query(Hawaii.tobs, Hawaii.date_format).\
                filter(Hawaii.name == 'WAIHEE 837.5, HI US').\
                filter(Hawaii.date_format.between('2016-08-23', '2017-08-23')).all()

In [73]:
df = pd.DataFrame(query_temp).set_index('date_format').rename(columns={'tobs': 'temperature_observations'})
df_dict = df.to_dict()

In [76]:
results = session.query(Hawaii.date_format, Hawaii.tobs).filter(Hawaii.date_format >= '2016-08-01', Hawaii.date_format <= '2017-08-31').all()
dic = pd.DataFrame(results).set_index('date_format').rename(columns={'tobs': 'temperature_observations'})
dic.temperature_observations = dic.temperature_observations.astype(float)
dic = dic.to_dict()

In [77]:
dic

{'temperature_observations': {Timestamp('2016-08-01 00:00:00'): 73.0,
  Timestamp('2016-08-02 00:00:00'): 72.0,
  Timestamp('2016-08-03 00:00:00'): 74.0,
  Timestamp('2016-08-04 00:00:00'): 74.0,
  Timestamp('2016-08-05 00:00:00'): 75.0,
  Timestamp('2016-08-06 00:00:00'): 77.0,
  Timestamp('2016-08-07 00:00:00'): 79.0,
  Timestamp('2016-08-08 00:00:00'): 75.0,
  Timestamp('2016-08-09 00:00:00'): 73.0,
  Timestamp('2016-08-10 00:00:00'): 76.0,
  Timestamp('2016-08-11 00:00:00'): 74.0,
  Timestamp('2016-08-12 00:00:00'): 71.0,
  Timestamp('2016-08-13 00:00:00'): 74.0,
  Timestamp('2016-08-14 00:00:00'): 72.0,
  Timestamp('2016-08-15 00:00:00'): 72.0,
  Timestamp('2016-08-16 00:00:00'): 72.0,
  Timestamp('2016-08-17 00:00:00'): 77.0,
  Timestamp('2016-08-18 00:00:00'): 73.0,
  Timestamp('2016-08-19 00:00:00'): 73.0,
  Timestamp('2016-08-20 00:00:00'): 76.0,
  Timestamp('2016-08-21 00:00:00'): 74.0,
  Timestamp('2016-08-22 00:00:00'): 72.0,
  Timestamp('2016-08-23 00:00:00'): 74.0,
  Time

In [76]:
str(query_temp_df.iloc[0]['date_format'])

'2016-08-23 00:00:00'

In [ ]:
#Query to take two dates and return all of the temp from the previous year from all statios

def get_temp(early_date_str, late_date_str):
    from datetime import datetime, timedelta
    early_date_dt = datetime.strptime(early_date_str, '%Y-%m-%d')
    late_date_dt = datetime.strptime(late_date_str, '%Y-%m-%d')
    early_date = early_date_dt + timedelta(days=-365)
    late_date = late_date_dt + timedelta(days=-365)
    ave_temp = session.query(func.avg(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    max_temp = session.query(func.max(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    min_temp = session.query(func.min(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    delta_temp = session.query(func.max(Hawaii.tobs) - func.min(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    return ave_temp, max_temp, min_temp, delta_temp

In [ ]:
get_temp('2017-08-01', '2017-08-12')

In [80]:
#Query to return one year's worth of prcp data.
#Function does a groupby for each day and sums across all four stations.

#https://stackoverflow.com/questions/25558582/groupby-and-sum-in-sqlalchemy
#https://stackoverflow.com/questions/8895208/sqlalchemy-how-to-filter-date-field

prcp_query = session.query(Hawaii.date_format, func.sum(Hawaii.prcp)).\
            filter(Hawaii.date_format.between('2016-08-23 00:00:00.000000', '2017-08-23 00:00:00.000000')).\
            group_by(Hawaii.date_format).first()

In [81]:
prcp_query

(datetime.datetime(2016, 8, 23, 0, 0), 2.7578461538461543)